# [Module 4.3.2] inference on custem tf serving image


---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r 

In [3]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

bert2tweet-2020-08-16-01-34-10-506


In [4]:
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Model Creation with Custome TFS Docker Image and Inference code

In [5]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [6]:
print(model.image)
print(model.serving_image_uri)
print(model.model_data)
print(model.name)
print(model.FRAMEWORK_NAME)


343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu
<bound method Model.serving_image_uri of <sagemaker.tensorflow.serving.Model object at 0x7f13848b0978>>
s3://sagemaker-ap-northeast-2-343441690612/bert2tweet-2020-08-16-01-34-10-506/output/model.tar.gz
None
tensorflow-serving


## 엔드포인트 생성

In [7]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


---------------!CPU times: user 1min 3s, sys: 11 s, total: 1min 14s
Wall time: 8min 40s


## Predictor Creation on the Endpoint

In [8]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-16-02-10-36-839


In [9]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [10]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [11]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
2860,i'm so happy you're here,0
6235,i'm praying for you to score good runs,4
2018,fr though especially you,5
8961,i'm really just tryna get turnt,8
7836,i honestly love seeing people i care about so ...,0
2754,young m a ooouuu what a banger,2
3551,i can't wait until i get proposed to like i ju...,5
544,dude yes i wanna spread my legs for errrrybody,3
2060,i been skinny my whole life,7
7617,i'm so ready take my hair out,6


In [12]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 3 이모티콘 추천

In [13]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
    print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),   
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    

        



reviews: 
 [" i'm so happy you're here"]
tweet:  i'm so happy you're here 
Ground_truth- 0:❤
 
Prediction: 1,💕,0,❤,6,😩,4,😊,7,😭 
 
reviews: 
 [" i'm praying for you to score good runs"]
tweet:  i'm praying for you to score good runs 
Ground_truth- 4:😊
 
Prediction: 7,😭,6,😩,1,💕,0,❤,3,😂 
 
reviews: 
 ['fr though especially you ']
tweet: fr though especially you  
Ground_truth- 5:😍
 
Prediction: 3,😂,7,😭,5,😍,4,😊,1,💕 
 
reviews: 
 ["i'm really just tryna get turnt"]
tweet: i'm really just tryna get turnt 
Ground_truth- 8:🙄
 
Prediction: 3,😂,6,😩,2,🔥,5,😍,8,🙄 
 
reviews: 
 ['i honestly love seeing people i care about so happy ']
tweet: i honestly love seeing people i care about so happy  
Ground_truth- 0:❤
 
Prediction: 1,💕,0,❤,6,😩,4,😊,9,🤔 
 
reviews: 
 ['young m a ooouuu what a banger']
tweet: young m a ooouuu what a banger 
Ground_truth- 2:🔥
 
Prediction: 3,😂,5,😍,6,😩,2,🔥,7,😭 
 
reviews: 
 ["i can't wait until i get proposed to like i just wonder how that moment will play out every day and i g

In [14]:
%store tweet_bert_endpoint_name

Stored 'tweet_bert_endpoint_name' (str)
